In [1]:
from ultralytics import YOLO
from lib.lib_utils import Utils 
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import cv2
import torch as Torch
import math
import os

In [2]:
#trasformazione .xyz to .png 
Utils.from_xyz_to_png(
    Path('/home/gabro/GrapheDetectProject/data.xyz/subset_xyz'), 
    Path('/home/gabro/GrapheDetectProject/'), 
    1)

generando .png del file graphene_238951.xyz...
1


In [ ]:
#divisione del dataset in train/ test/
#inserire percorso dataset da dividere e percentuale del test 
Utils.split_dataset('/home/gabro/GrapheDetectProject/data_yolo_new/', 0.2)   

In [15]:
# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("/home/gabro/GrapheDetectProject/best_100_campioni_new.pt")  # load a pretrained model (recommended for training)


In [ ]:
# Use the model
# model.train(data="coco128.yaml", epochs=3)  # train the model
model.train(data="config_yolov8.yaml", epochs=100)  # train the model

In [3]:
metrics = model.val()  # evaluate model performance on the validation set
results = model("/home/gabro/GrapheDetectProject/b74c9ce2-graphene_218641_bonds.png")  # predict on an image
success = model.export(format="onnx")  # export the model to ONNX format

Ultralytics YOLOv8.0.88 🚀 Python-3.11.3 torch-2.0.0+cu117 CPU
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


FileNotFoundError: '/content/gdrive/MyDrive/Materiale tirocinio Gabriele/GrapheDefectDetector/google_colab_config.yaml' does not exist

In [19]:
Utils.funzione_prova()
Utils.crop_from_folder("/home/gabro/GrapheDetectProject/immProvaInferenza", "/home/gabro/GrapheDetectProject/cartellaCrop", model)
Utils.from_crops_to_thresh('/home/gabro/GrapheDetectProject/cartellaCrop/')

ok!


AttributeError: type object 'Utils' has no attribute 'from_crops_to_thresh'

Inferenza e trasformazione boxes in immagini singole 

In [40]:
#crop su immagine singola
pathImmProva = "/home/gabro/GrapheDetectProject/b74c9ce2-graphene_218641_bonds.png"
results = model(pathImmProva)  # predict on an image

boxes = results[0].boxes
index = 0
print(boxes.data.size)
# for box in boxes.data:
#     print(box.xyxy)
for box in boxes:
    #devo trasformare xy (alto sx), xy (alto dx) in [start_row:end_row, start_col:end_col]
    print(boxes[index].xyxy)    #posizione angolo in alto a sx e in basso a dx in pixel
    array = Torch.Tensor.numpy(boxes[index].xyxy)
    # print(array.size)
    x1 = math.ceil(array[0,0])
    y1 = math.ceil(array[0,1])
    x2 = math.floor(array[0,2])
    y2 = math.floor(array[0,3])

    # res_plotted = results[0].plot(labels = False, line_width = 1)
    # plt.imshow(res_plotted)
    # plt.show()
    # cv2.imshow("result", res_plotted)

    #crop difetto
    img = cv2.imread(pathImmProva)
    print(img.shape) # Print image shape
    # cv2.imshow("original", img)
    
    # Cropping an image
    cropped_image = img[y1:y2, x1:x2] #img[start_row:end_row, start_col:end_col]

    # Display cropped image
    # cv2.imshow("cropped", cropped_image)

    nameCropped = pathImmProva.removesuffix('.png') + "_cropped_box_" + str(index)+ ".png"
    # Save the cropped image
    cv2.imwrite(nameCropped, cropped_image)
    
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    index = index+1




image 1/1 /home/gabro/GrapheDetectProject/b74c9ce2-graphene_218641_bonds.png: 640x640 2 defects, 278.0ms
Speed: 2.5ms preprocess, 278.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


<built-in method size of Tensor object at 0x7fac5848e530>
tensor([[ 4.3943, 18.4342, 34.3378, 50.7019]])
(69, 70, 3)
tensor([[37.1617, 33.1211, 59.6014, 56.4856]])
(69, 70, 3)


In [6]:
#crop_from_folder
#prende una cartella di immagini su cui fare detection dei difetti,
#deposita su una cartella i difetti croppati 

#versione con for al posto di .predict
cartellaImm = "/home/gabro/GrapheDetectProject/immProvaInferenza/"

cartellaCrop = "/home/gabro/GrapheDetectProject/cartellaCrop/"  #cartella dove andranno messi i difetti croppati 


for nomeImm in os.listdir(cartellaImm): #ietro i nomi delle immagini all'interno della cartella

    pathImm = os.path.join(cartellaImm, nomeImm)    #estraggo il path dell'immagine 
    results = model(pathImm)  # predict on an image
    boxes = results[0].boxes
    index = 0
    print(boxes.data.size)
    # for box in boxes.data:
    #     print(box.xyxy)
    for box in boxes:
        #devo trasformare xy (alto sx), xy (alto dx) in [start_row:end_row, start_col:end_col]
        print(boxes[index].xyxy)    #posizione angolo in alto a sx e in basso a dx in pixel
        array = Torch.Tensor.numpy(boxes[index].xyxy)
        # print(array.size)
        x1 = math.floor(array[0,0])
        y1 = math.floor(array[0,1])
        x2 = math.ceil(array[0,2])
        y2 = math.ceil(array[0,3])

        # res_plotted = results[0].plot(labels = False, line_width = 1)
        # plt.imshow(res_plotted)
        # plt.show()
        # cv2.imshow("result", res_plotted)

        #crop difetto
        img = cv2.imread(pathImm)
        print(img.shape) # Print image shape
        # cv2.imshow("original", img)
        
        # Cropping an image
        cropped_image = img[y1:y2, x1:x2] #img[start_row:end_row, start_col:end_col]

        # Display cropped image
        # cv2.imshow("cropped", cropped_image)

        nomeCropped = os.path.basename(pathImm).removesuffix('.png') + "_cropped_box_" + str(index)+ ".png" #creo nome dell'immagine croppata
        pathCropped = os.path.join(cartellaCrop, nomeCropped)   #compongo il path dell'immagine croppata aggiungendo il giusto percorso di output
        # Save the cropped image
        cv2.imwrite(pathCropped, cropped_image)
        
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        index = index+1


image 1/1 /home/gabro/GrapheDetectProject/immProvaInferenza/0e9ff1ff-graphene_176095_bonds.png: 640x640 2 defects, 546.3ms
Speed: 5.5ms preprocess, 546.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)



<built-in method size of Tensor object at 0x7f63f9812080>
tensor([[38.5175, 39.5588, 51.7656, 52.2145]])
(69, 70, 3)
tensor([[ 3.4684, 16.6030, 41.2881, 53.1532]])
(69, 70, 3)


image 1/1 /home/gabro/GrapheDetectProject/immProvaInferenza/0a288a23-graphene_227444_bonds.png: 640x640 2 defects, 541.7ms
Speed: 1.4ms preprocess, 541.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)



<built-in method size of Tensor object at 0x7f63f96b8050>
tensor([[22.3563, 30.9228, 51.8132, 53.0598]])
(69, 70, 3)
tensor([[ 0.0000, 18.6053, 17.3393, 35.6328]])
(69, 70, 3)


image 1/1 /home/gabro/GrapheDetectProject/immProvaInferenza/2d81c496-graphene_122634_bonds.png: 640x640 2 defects, 383.3ms
Speed: 3.7ms preprocess, 383.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)



...

<built-in method size of Tensor object at 0x7f63f98620d0>
tensor([[34.9728,  0.5022, 49.4850, 13.9389]])
(69, 70, 3)


image 1/1 /home/gabro/GrapheDetectProject/immProvaInferenza/2f01675e-graphene_231617_bonds.png: 640x640 1 defect, 385.5ms
Speed: 5.3ms preprocess, 385.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



<built-in method size of Tensor object at 0x7f63f96bb340>
tensor([[25.0373, 23.2366, 56.0320, 60.3772]])
(69, 70, 3)


image 1/1 /home/gabro/GrapheDetectProject/immProvaInferenza/2c60508d-graphene_312780_bonds.png: 640x640 1 defect, 421.3ms
Speed: 4.9ms preprocess, 421.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


<built-in method size of Tensor object at 0x7f63f96e3d40>
tensor([[21.1721, 36.0090, 54.6227, 68.6001]])
(69, 70, 3)


In [23]:
#non funziona (usare codice in alto)
cartellaImm = "/home/gabro/GrapheDetectProject/immProvaInferenza"
results = model(cartellaImm, save = False)  # predict on an image

for (r, nomeImm) in zip(results, os.listdir(cartellaImm)):
    pathImm = os.path.join(cartellaImm, nomeImm)
    print(pathImm)

    boxes = r.boxes
    index = 0
    print(boxes.data.size)
    # for box in boxes.data:
    #     print(box.xyxy)
    for box in boxes:
        #devo trasformare xy (alto sx), xy (alto dx) in [start_row:end_row, start_col:end_col]
        print(box.xyxy)    #posizione angolo in alto a sx e in basso a dx in pixel
        array = Torch.Tensor.numpy(box.xyxy)
        # print(array.size)
        x1 = math.ceil(array[0,0])
        y1 = math.ceil(array[0,1])
        x2 = math.floor(array[0,2])
        y2 = math.floor(array[0,3])

        # res_plotted = results[0].plot(labels = False, line_width = 1)
        # plt.imshow(res_plotted)
        # plt.show()
        # cv2.imshow("result", res_plotted)

        #crop difetto
        img = cv2.imread(pathImm)
        # print(img.shape) # Print image shape
        # cv2.imshow("original", img)
        
        # Cropping an image
        cropped_image = img[y1:y2, x1:x2] #img[start_row:end_row, start_col:end_col]

        # Display cropped image
        # cv2.imshow("cropped", cropped_image)

        nameCropped = pathImm.removesuffix('.png') + "_cropped_box_" + str(index)+ ".png"
        # Save the cropped image
        cv2.imwrite(nameCropped, cropped_image)
        
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        index = index+1



image 1/6 /home/gabro/GrapheDetectProject/immProvaInferenza/0a288a23-graphene_227444_bonds.png: 640x640 2 defects, 288.6ms
image 2/6 /home/gabro/GrapheDetectProject/immProvaInferenza/0a5c3409-graphene_301236_bonds.png: 640x640 1 defect, 236.9ms
image 3/6 /home/gabro/GrapheDetectProject/immProvaInferenza/0e9ff1ff-graphene_176095_bonds.png: 640x640 2 defects, 301.1ms
image 4/6 /home/gabro/GrapheDetectProject/immProvaInferenza/2c60508d-graphene_312780_bonds.png: 640x640 1 defect, 300.2ms
image 5/6 /home/gabro/GrapheDetectProject/immProvaInferenza/2d81c496-graphene_122634_bonds.png: 640x640 2 defects, 302.2ms
image 6/6 /home/gabro/GrapheDetectProject/immProvaInferenza/2f01675e-graphene_231617_bonds.png: 640x640 1 defect, 274.1ms
Speed: 1.8ms preprocess, 283.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict2


/home/gabro/GrapheDetectProject/immProvaInferenza/0e9ff1ff-graphene_176095_bonds.png
<built-in method size of Tensor object at 0x7f24532e2cb0>
tensor([[22.3563, 30.9228, 51.8132, 53.0598]])
tensor([[ 0.0000, 18.6053, 17.3393, 35.6328]])
/home/gabro/GrapheDetectProject/immProvaInferenza/0a288a23-graphene_227444_bonds.png
<built-in method size of Tensor object at 0x7f24532e3f70>
tensor([[34.9728,  0.5022, 49.4850, 13.9389]])
/home/gabro/GrapheDetectProject/immProvaInferenza/2d81c496-graphene_122634_bonds.png
<built-in method size of Tensor object at 0x7f24532e1fe0>
tensor([[38.5175, 39.5588, 51.7656, 52.2145]])
tensor([[ 3.4684, 16.6030, 41.2881, 53.1532]])
/home/gabro/GrapheDetectProject/immProvaInferenza/0a5c3409-graphene_301236_bonds.png
<built-in method size of Tensor object at 0x7f24532e1d60>
tensor([[21.1721, 36.0090, 54.6227, 68.6001]])
/home/gabro/GrapheDetectProject/immProvaInferenza/2f01675e-graphene_231617_bonds.png
<built-in method size of Tensor object at 0x7f24532e32a0>
ten

In [14]:
#fa inferenza su una cartella di immagini e ne salva i crop su runs/detect/predict#/crop/defect
pathCartellaImm = "/home/gabro/GrapheDetectProject/immProvaInferenza/"
results = model.predict(pathCartellaImm, save_crop = True')



SyntaxError: '[31m[1msave_dir[0m' is not a valid YOLO argument. Similar arguments are i.e. ['save_crop=False', 'save=True', 'save_period=-1'].

    Arguments received: ['yolo', '--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"0345ed87-9ffd-412a-81c1-86603fceaaff"', '--shell=9002', '--transport="tcp"', '--iopub=9004', '--f=/home/gabro/.local/share/jupyter/runtime/kernel-v2-2901WgmI7MIVlul5.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ('detect', 'segment', 'classify', 'pose')
                MODE (required) is one of ('train', 'val', 'predict', 'export', 'track', 'benchmark')
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 e